## Code to compare lineage-specific responsiveness [after treatment with external stressor (osteogenic differentiation medium)] in high D cells (FLAT) with low D cells (PILLAR) 

### Written by Ranya K. A. Virk & Vasundhara Agrawal

In [ ]:
#Import all libraries
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import os,sys,glob
import pandas as pd
import time
from sympy.solvers import solve
from sympy import Symbol
import sympy as sp
import scipy as sc
from scipy import stats
from scipy.stats import binned_statistic
from matplotlib.ticker import MaxNLocator
from scipy.stats import norm
from matplotlib.ticker import FormatStrFormatter
import scipy.optimize 
import seaborn as sns
from scipy.interpolate import make_interp_spline, BSpline
from mpl_toolkits import mplot3d
from matplotlib import rcParams
from IPython.display import display


## Define Function Calculating Sensitivity (CPMC Model)

In [ ]:
##################################################################################
#### Calculate the change of expression due to the change of chromatin packing (CPMC)###
# This Code has been previously been made available on GitHub: https://github.com/BiophotonicsNU/macrogenomics.
##################################################################################

def Se(D_fit):
    '''
    D_fit is the chromatin packing scaling
    '''
  
    
    ## Load MC simulation result
    sec          =np.genfromtxt('second_derivative_TF_norm.csv')
    mRNA_initial =np.genfromtxt('max_mRNA_initial.csv')
    phi_initial  =np.genfromtxt('phi_initial.csv')
    tot_con      =np.genfromtxt('tot_con.csv')
    Li0          =15e-9 # Size of interaction volume for single base pair
    r_min        =1e-9 # size of elementary particle
    len_gene     =6e3 # average length of genes
    initial_aver =0.00056 # the average initial expression rate
    M_f = 1e6
    Li           =(Li0+len_gene**(1/D_fit)*r_min) # the total interaction volume

    ## Load the equation of Se
    ratio = M_f#(r_max/r_min)**D_fit # M_f/M_min
    sigma = phi_initial*(1-phi_initial) # variance of crowding density at each point
    sigma_phi = sigma*(Li/r_min)**(D_fit-3) # variance of average crowding density in each interaction volume 
    d_sigma_phi = sigma_phi*(np.log(Li/r_min)+(3-D_fit)/D_fit**2*len_gene**(1/D_fit)*np.log(len_gene)*(r_min/Li)) # the derivative of sigma_phi as respect to D
    Se = (1/D_fit**2*np.log(ratio)+1/2*sec/(1+1/2*sec*sigma_phi)*d_sigma_phi)*D_fit # gene transcription Sensitivity
    initial_expression = initial_aver 
    expression = mRNA_initial*(1+1/2*sec*sigma_phi) # the average expression of gene share similar molecular properties
    ## Fitting the model with poly10
    x = np.log(expression/initial_aver)
    y = Se
    index_start=np.arange(x.shape[0])[~np.isnan(x)][0]
    para2 = np.poly1d(np.polyfit(x[index_start:],y[index_start:],10))
    x = np.linspace(-5,3,100) ## (-3.8,3) for 3D plot. (-5,3) for cross-section
    return (np.exp(x)*initial_aver,para2(x))

## All DE Genes in Induced MSCs compared to Control MSCs (p-adj<0.05)
### Evaluating Experimental Results and CPMC Model Predictions:

In [ ]:
fig, ax = plt.subplots(figsize = (15,11))

##################################################################################
#### Experimental Calculation of Lineage Specific Responsiveness Coefficient #####
##################################################################################

#Treatement groups: SF = MSCs on Flat; SP = MSCs on Pillars; IF = Induced MSCs on Flat; IP = Induced MSCs on Pillars
#Load data (Differnetially expressed genes with average gene expression for each group, p<0.05):
ALL_reg_genes = pd.read_csv('DE_Meanvalues.csv')
ALL_mean_genes = pd.DataFrame(index = ALL_reg_genes.index, columns = ['IF','IP','SF','SP'])
ALL_mean_genes.IF = ALL_reg_genes ['IF']
ALL_mean_genes.SF = ALL_reg_genes ['SF']
ALL_mean_genes.IP = ALL_reg_genes ['IP']
ALL_mean_genes.SP = ALL_reg_genes ['SP']

#Expression Relative to control (uninduced MSCs on Flat):
ALL_expression_norm=(np.log2(ALL_mean_genes.div(ALL_mean_genes['SF'],axis=0)))

## Divide control into 2 quantile subgroups based on initial expression of stem state:
lb=['Experiment\nLow Initial Expression','Experiment\nHigh Initial Expression']
quant_con = np.percentile(ALL_mean_genes['SF'],[12,50,60,98])
color_index=[1,4]
for i in range(2):
    
    display(i)
    gene_con = (ALL_mean_genes['SF']<quant_con[i*2+1]) & (ALL_mean_genes['SF']>quant_con[i*2])
    expression_con_norm= ALL_expression_norm[gene_con]
    expression_con=ALL_mean_genes[gene_con]
    
    ## divide induced MSCs into decile subgroups
    quant_inducedMSCs = np.percentile(expression_con_norm['IF'],np.linspace(0,100,11))
    y = []
    error=[]

    for j in range(10):
        gene = (expression_con_norm['IF']<quant_inducedMSCs[j+1]) & (expression_con_norm['IF']>quant_inducedMSCs[j])
        de = expression_con['IF'][gene]/expression_con['SF'][gene]
        nu = expression_con['IP'][gene]/expression_con['SP'][gene]
        cov=np.cov(nu,de)[1,0] 
        
        ##### Evaluate lineage-specific reponsiveness as the ratio of expression rate of (IP/SP)/(IF/SF):
        y.append(np.mean(nu)/np.mean(de))
        error.append((np.mean(nu)/np.mean(de))*(np.var(nu)/np.mean(nu)**2+np.var(de)/np.mean(de)**2\
                     -2*cov/np.mean(de)/np.mean(nu))**(1/2)/np.size(nu)**(1/2))
    ax.errorbar(quant_inducedMSCs[1:],y,markersize=25,yerr=error,fmt='.',capsize=7,label=lb[i],color=sns.color_palette("muted")[color_index[i]])
    ax.set_ylabel(r'$R_{LS}$',fontsize=38)
    ax.set_xlabel(r'$log_2(E_{IF}/E_{SF})$',fontsize=38,weight='bold')

##################################################################################
##### CPMC Model Prediction of Lineage Specific Responsiveness Coefficient #######
##################################################################################

#Range of D measured in MSCs:
D_0 = 2.85                        
D_i = np.linspace(D_0,2.58,100)  
dD =D_i[1]-D_i[0]

# Calculate CPMC predicted sensitivity (dlnE/dlnD) for each D value in given range (3D array)
Se_all=[]
for i in range(100):
    temp,Se_temp=Se(D_i[i])
    Se_all.append(Se_temp)
inte=np.exp((((np.array(Se_all).transpose()/D_i).transpose())*dD).sum(0)) 

lb=['CPMC Model\nLow Initial Expression','CPMC Model\nHigh Initial Expression']
color_index=[1,4]
x,temp = Se(D_0)
y,temp = Se(D_0)
ind=[70,85] # Indices representing the relative initial expression of the stem flat state for initially lowly expressed
            # and initially highly expressed genes, used to subset 2D slices from 3D sensitivity curve
xlim=[-2,5,-2,5]  
for j,i in enumerate(ind):
    # Calculate predicted expression rate for lower D chromatin
    b_1_0 = x[i] 
    b_2_0 = y
    db_0= b_2_0/b_1_0

    # Calculate predicted expression for higher D chromatin
    x_high_D=x[i]*inte[i]
    y_high_D= y*inte
    db=y_high_D/x_high_D
    
    # Calculate predicted lineage specific responsiveness coefficient 
    b_rate = db_0/db
    
    # Plot CPMC predictions within a given range of np.log2(b_2_0/b_1_0)
    x_plot=np.log2(b_2_0/b_1_0)
    y_plot=1/b_rate
    ind=(x_plot>xlim[j*2])&(x_plot<=xlim[j*2+1])
    x_plot=x_plot[ind]
    y_plot=y_plot[ind]
    ax.plot(x_plot, y_plot,label=lb[j],c=sns.color_palette('muted')[color_index[j]])

ax.legend(frameon=True,fontsize=20)

ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)


ax.set_xlim(-2.5,6)
ax.set_ylim(0.5,1.6)
plt.autoscale(False)
ax.grid(False)
for axis in ['bottom','right']:
    ax.spines[axis].set_linewidth(3)
    ax.spines[axis].set_color('k')

ax.set_facecolor('white')
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
plt.xticks(fontsize=29,weight='bold')
plt.yticks(fontsize=29,weight='bold')
fig.savefig('ModelA-2.85-2.58_70-85_'+str(ind[0])+'_'+str(ind[1])+'.tif',dpi=1000)

